In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import warnings
import time
import re
warnings.filterwarnings(action='ignore')

In [5]:
# 드라이버 불러오기
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [29]:
# 메인 페이지
url = 'https://biz.koke.kr/products?filters=%7B%22categoryGroup%22%3A%22WholeBean%22%7D'
driver.get(url) # go to website
time.sleep(1)

In [11]:
links = [] # 제품 링크 리스트

# 약 4분 30초 소요
for i in range(32):
    raw = driver.find_elements(By.CSS_SELECTOR,"div.flex.flex-col.w-full.lg\:w-3\/4")
    link_elements = raw[0].find_elements(By.TAG_NAME,'a') 
    href_links = [link.get_attribute('href') for link in link_elements]
    links = links + href_links # 현재 페이지에 있는 상품 가져오기
    time.sleep(3)

    if i is not 31:
        button = driver.find_elements(By.CSS_SELECTOR ,"button.text-gray-500.bg-white.border.border-gray-200.koke-button")
        button[-1].click()
        time.sleep(5)
    
    print('현제 페이지(i+1): ',i+1,' 링크 수: ',len(href_links),' 총 링크 수: ',len(links))

link = set(links)
print(len(links)) # 총 556개


현제 페이지(i+1):  1  링크 수:  18  총 링크 수:  18
현제 페이지(i+1):  2  링크 수:  18  총 링크 수:  36
현제 페이지(i+1):  3  링크 수:  18  총 링크 수:  54
현제 페이지(i+1):  4  링크 수:  18  총 링크 수:  72
현제 페이지(i+1):  5  링크 수:  18  총 링크 수:  90
현제 페이지(i+1):  6  링크 수:  18  총 링크 수:  108
현제 페이지(i+1):  7  링크 수:  18  총 링크 수:  126
현제 페이지(i+1):  8  링크 수:  18  총 링크 수:  144
현제 페이지(i+1):  9  링크 수:  18  총 링크 수:  162
현제 페이지(i+1):  10  링크 수:  18  총 링크 수:  180
현제 페이지(i+1):  11  링크 수:  18  총 링크 수:  198
현제 페이지(i+1):  12  링크 수:  18  총 링크 수:  216
현제 페이지(i+1):  13  링크 수:  18  총 링크 수:  234
현제 페이지(i+1):  14  링크 수:  18  총 링크 수:  252
현제 페이지(i+1):  15  링크 수:  18  총 링크 수:  270
현제 페이지(i+1):  16  링크 수:  18  총 링크 수:  288
현제 페이지(i+1):  17  링크 수:  18  총 링크 수:  306
현제 페이지(i+1):  18  링크 수:  18  총 링크 수:  324
현제 페이지(i+1):  19  링크 수:  18  총 링크 수:  342
현제 페이지(i+1):  20  링크 수:  18  총 링크 수:  360
현제 페이지(i+1):  21  링크 수:  18  총 링크 수:  378
현제 페이지(i+1):  22  링크 수:  18  총 링크 수:  396
현제 페이지(i+1):  23  링크 수:  18  총 링크 수:  414
현제 페이지(i+1):  24  링크 수:  18  총 링크 수:  432
현제 페이지

In [14]:
import pickle

# Pickle dump the set of links to a file
with open("links_559.pkl", "wb") as file:
    pickle.dump(link, file)

# Load the set of links from the file
with open("links_559.pkl", "rb") as file:
    loaded_links = pickle.load(file)

# Print the loaded set of links
print("Loaded links:", len(loaded_links))

Loaded links: 559


In [31]:
def get_roastery(url):
    # 제품 사이트로 이동
    driver.get(url)
    time.sleep(3)
    # 정보 블록
    raw = driver.find_element(By.CSS_SELECTOR,"div.md\:pt-\[4rem\]")
    
    # 원두 ID
    pattern = r"https://biz.koke.kr/products/(\d+)" 
    match = re.search(pattern, url)
    if match:
        product_number = match.group(1)
        id = product_number  
    # 원두 이름
    bean_name = driver.find_element(By.CSS_SELECTOR,"div.flex.flex-col").find_element(By.TAG_NAME,'h1').text
    # 로스터리 이름
    roastery = raw.find_element(By.TAG_NAME,'h2').text
    # 소개글
    temp = raw.find_element(By.CSS_SELECTOR,"div.w-full.md\:w-168.md\:koke-word-keep-all")
    introduction = temp.find_element(By.TAG_NAME,'p').text
    # 주소
    address = temp.find_elements(By.TAG_NAME,'li')[-1].find_element(By.TAG_NAME,'p').text
    # 이미지
    img = raw.find_elements(By.TAG_NAME,'img')[-1].get_attribute('src')
    
    return [id,bean_name,roastery,introduction,address,img]


In [51]:
temp_list = []
for url in list(link)[366:]:
    temp = get_roastery(url)
    temp_list.append(temp)
    print(i,'번째 자료수집중...')
    i+=1

367 번째 자료수집중...
368 번째 자료수집중...
369 번째 자료수집중...
370 번째 자료수집중...
371 번째 자료수집중...
372 번째 자료수집중...
373 번째 자료수집중...
374 번째 자료수집중...
375 번째 자료수집중...
376 번째 자료수집중...
377 번째 자료수집중...
378 번째 자료수집중...
379 번째 자료수집중...
380 번째 자료수집중...
381 번째 자료수집중...
382 번째 자료수집중...
383 번째 자료수집중...
384 번째 자료수집중...
385 번째 자료수집중...
386 번째 자료수집중...
387 번째 자료수집중...
388 번째 자료수집중...
389 번째 자료수집중...
390 번째 자료수집중...
391 번째 자료수집중...
392 번째 자료수집중...
393 번째 자료수집중...
394 번째 자료수집중...
395 번째 자료수집중...
396 번째 자료수집중...
397 번째 자료수집중...
398 번째 자료수집중...
399 번째 자료수집중...
400 번째 자료수집중...
401 번째 자료수집중...
402 번째 자료수집중...
403 번째 자료수집중...
404 번째 자료수집중...
405 번째 자료수집중...
406 번째 자료수집중...
407 번째 자료수집중...
408 번째 자료수집중...
409 번째 자료수집중...
410 번째 자료수집중...
411 번째 자료수집중...
412 번째 자료수집중...
413 번째 자료수집중...
414 번째 자료수집중...
415 번째 자료수집중...
416 번째 자료수집중...
417 번째 자료수집중...
418 번째 자료수집중...
419 번째 자료수집중...
420 번째 자료수집중...
421 번째 자료수집중...
422 번째 자료수집중...
423 번째 자료수집중...
424 번째 자료수집중...
425 번째 자료수집중...
426 번째 자료수집중...
427 번째 자료수집중...
428 번째 자료수집중...
429 번째 자

In [55]:
df = pd.DataFrame(temp_list,
                  columns=['id','name','roastery','info','address','img'])
df.head()

,id,name,roastery,info,address,img
0,43,데일리스윗,언더프레셔,“일상 속 특별한 휴식”\n커피를 추출하는 기본 원리인 물줄기의 압력(Pressur...,서울 강남구 영동대로106길 37 1층,https://d2wosiipoa41qn.cloudfront.net/RVgOLNI3...
1,3896,브라질 세라도 디카페인,운조커피,커피가 나에게 준 커다란 변화를 모두와 나누고 싶습니다.\n나의 이름이 붙은 커피를...,경기도 화성시 동탄지성로 306번길 68,https://d2wosiipoa41qn.cloudfront.net/MOJEAUd2...
2,3458,에티오피아 예가체프 아리차 g1 내추럴,제이엠디커피팩토리,“JMD만의 명품 원두로 그 이상의 무한함을 창조하고자 합니다.”\n커피는 이제 단...,경기 고양시 일산동구 장대길 75-53 나동 1층 JMD Coffee Factory,https://d2wosiipoa41qn.cloudfront.net/yigVBzLf...
3,22,1st WAVE : Supply,뉴웨이브 커피로스터스,뉴웨이브 커피로스터스는 확고한 커피의 철학으로 우리만의 스타일로 많은 커피를 전달하...,"서울특별시 양천구 오목로56길 2,1층",https://d2wosiipoa41qn.cloudfront.net/tn_QSSI1...
4,180,에티오피아 리무 볼렌소,베르크 로스터스,"부산에 위치한 스몰배치 스페셜티 커피 로스터리이자, 서로 다른 경험과 개성을 가진 ...",부산 부산진구 전포동 356-6 지하1층,https://d2wosiipoa41qn.cloudfront.net/ZCJvGQcr...


In [56]:
df.to_csv('roastery.csv')

# 로스터리 이미지 다운로드


In [57]:
df = pd.read_csv('roastery.csv').iloc[:,1:]
df.head()

,id,name,roastery,info,address,img
0,43,데일리스윗,언더프레셔,“일상 속 특별한 휴식”\n커피를 추출하는 기본 원리인 물줄기의 압력(Pressur...,서울 강남구 영동대로106길 37 1층,https://d2wosiipoa41qn.cloudfront.net/RVgOLNI3...
1,3896,브라질 세라도 디카페인,운조커피,커피가 나에게 준 커다란 변화를 모두와 나누고 싶습니다.\n나의 이름이 붙은 커피를...,경기도 화성시 동탄지성로 306번길 68,https://d2wosiipoa41qn.cloudfront.net/MOJEAUd2...
2,3458,에티오피아 예가체프 아리차 g1 내추럴,제이엠디커피팩토리,“JMD만의 명품 원두로 그 이상의 무한함을 창조하고자 합니다.”\n커피는 이제 단...,경기 고양시 일산동구 장대길 75-53 나동 1층 JMD Coffee Factory,https://d2wosiipoa41qn.cloudfront.net/yigVBzLf...
3,22,1st WAVE : Supply,뉴웨이브 커피로스터스,뉴웨이브 커피로스터스는 확고한 커피의 철학으로 우리만의 스타일로 많은 커피를 전달하...,"서울특별시 양천구 오목로56길 2,1층",https://d2wosiipoa41qn.cloudfront.net/tn_QSSI1...
4,180,에티오피아 리무 볼렌소,베르크 로스터스,"부산에 위치한 스몰배치 스페셜티 커피 로스터리이자, 서로 다른 경험과 개성을 가진 ...",부산 부산진구 전포동 356-6 지하1층,https://d2wosiipoa41qn.cloudfront.net/ZCJvGQcr...


In [58]:
filtered_df = df.drop_duplicates(subset='roastery', keep='first').loc[:,['roastery','img']].reset_index().iloc[:,1:]
filtered_df.head()

,roastery,img
0,언더프레셔,https://d2wosiipoa41qn.cloudfront.net/RVgOLNI3...
1,운조커피,https://d2wosiipoa41qn.cloudfront.net/MOJEAUd2...
2,제이엠디커피팩토리,https://d2wosiipoa41qn.cloudfront.net/yigVBzLf...
3,뉴웨이브 커피로스터스,https://d2wosiipoa41qn.cloudfront.net/tn_QSSI1...
4,베르크 로스터스,https://d2wosiipoa41qn.cloudfront.net/ZCJvGQcr...


In [62]:
import requests

for i in range(len(filtered_df)):
    roastery = filtered_df.loc[i,'roastery']
    img_url = filtered_df.loc[i,'img']
    
    response = requests.get(img_url)
    
    if response.status_code == 200:
        with open('roastery_img/'+str(roastery)+'.jpg', 'wb') as file:
            file.write(response.content)
            print(i,'번째 ',roastery, " Image downloaded successfully.")
    else:
        print(roastery, f" Failed to download the image. Status code: {response.status_code}")


0 번째  언더프레셔  Image downloaded successfully.
1 번째  운조커피  Image downloaded successfully.
2 번째  제이엠디커피팩토리  Image downloaded successfully.
3 번째  뉴웨이브 커피로스터스  Image downloaded successfully.
4 번째  베르크 로스터스  Image downloaded successfully.
5 번째  조바티커피  Image downloaded successfully.
6 번째  라메르판지 비앤알  Image downloaded successfully.
7 번째  토치커피  Image downloaded successfully.
8 번째  리플로우 커피 로스터스  Image downloaded successfully.
9 번째  커스프 커피  Image downloaded successfully.
10 번째  나무사이로  Image downloaded successfully.
11 번째  전광수 커피  Image downloaded successfully.
12 번째  오프사이트  Image downloaded successfully.
13 번째  수수커피  Image downloaded successfully.
14 번째  그린마일커피  Image downloaded successfully.
15 번째  커피네츄럴스페셜티  Image downloaded successfully.
16 번째  폰트커피  Image downloaded successfully.
17 번째  프레이저 커피  Image downloaded successfully.
18 번째  180커피로스터스  Image downloaded successfully.
19 번째  피어 커피로스터스  Image downloaded successfully.
20 번째  히떼 로스터리  Image downloaded successfully.
21 번째  스티머스 커피팩토리  Image d